In [3]:
# importing libraries

import pandas as pd
import numpy as np
import seaborn as sns
import regex as re


In [4]:
# reading into the sheets in main excel file

transactions = pd.read_excel('KPMG_Datasets.xlsx', sheet_name=0)
new_customer_list = pd.read_excel('KPMG_Datasets.xlsx', sheet_name=1)
customer_demographic = pd.read_excel('KPMG_Datasets.xlsx', sheet_name=2)
customer_address = pd.read_excel('KPMG_Datasets.xlsx', sheet_name=3)

Data Validation
    - check for:
        1. consistency: numerical (are there any outliers) object ( M
        and F, f and m, or Male and Female)

        2. accuracy: if the data in the col is the correct type

        3. completeness: are there null values

        4. relevancy: if the data collected matches what the col is and what the dataset is about


____________ Transactions Dataset ____________

In [4]:
# Preliminary data validation
print(transactions.head(), '\n\n')
print(transactions.info(), '\n\n')
print(transactions.isnull().sum())


   transaction_id  product_id  customer_id transaction_date  online_order  \
0               1           2         2950       2017-02-25           0.0   
1               2           3         3120       2017-05-21           1.0   
2               3          37          402       2017-10-16           0.0   
3               4          88         3135       2017-08-31           0.0   
4               5          78          787       2017-10-01           1.0   

  order_status           brand product_line product_class product_size  \
0     Approved           Solex     Standard        medium       medium   
1     Approved   Trek Bicycles     Standard        medium        large   
2     Approved      OHM Cycles     Standard           low       medium   
3     Approved  Norco Bicycles     Standard        medium       medium   
4     Approved  Giant Bicycles     Standard        medium        large   

   list_price  standard_cost  product_first_sold_date  
0       71.49          53.62        

In [44]:
print(transactions.describe())

       transaction_id   product_id   customer_id            transaction_date  \
count    20000.000000  20000.00000  20000.000000                       20000   
mean     10000.500000     45.36465   1738.246050  2017-07-01 14:08:05.280000   
min          1.000000      0.00000      1.000000         2017-01-01 00:00:00   
25%       5000.750000     18.00000    857.750000         2017-04-01 00:00:00   
50%      10000.500000     44.00000   1736.000000         2017-07-03 00:00:00   
75%      15000.250000     72.00000   2613.000000         2017-10-02 00:00:00   
max      20000.000000    100.00000   5034.000000         2017-12-30 00:00:00   
std       5773.647028     30.75359   1011.951046                         NaN   

       online_order    list_price  standard_cost  product_first_sold_date  
count  19640.000000  20000.000000   19803.000000             19803.000000  
mean       0.500458   1107.829449     556.046951             38199.776549  
min        0.000000     12.010000       7.210000   

In [5]:
# Consistency: among object columns

object_cols = ['order_status', 'brand', 'product_line', 'product_class', 'product_size']

    # do not directly use value_counts on a df - will overload system

for column in object_cols:
    print(transactions[column].value_counts(), '\n')

    # observe all of these columns are clean in terms of consistency
        # uniform strings in all of these columns

order_status
Approved     19821
Cancelled      179
Name: count, dtype: int64 

brand
Solex             4253
Giant Bicycles    3312
WeareA2B          3295
OHM Cycles        3043
Trek Bicycles     2990
Norco Bicycles    2910
Name: count, dtype: int64 

product_line
Standard    14176
Road         3970
Touring      1234
Mountain      423
Name: count, dtype: int64 

product_class
medium    13826
high       3013
low        2964
Name: count, dtype: int64 

product_size
medium    12990
large      3976
small      2837
Name: count, dtype: int64 



In [6]:
# Consistency: among numerical columns

print(transactions.describe())

    # check for outliers
        # no negative values for price col

       transaction_id   product_id   customer_id            transaction_date  \
count    20000.000000  20000.00000  20000.000000                       20000   
mean     10000.500000     45.36465   1738.246050  2017-07-01 14:08:05.280000   
min          1.000000      0.00000      1.000000         2017-01-01 00:00:00   
25%       5000.750000     18.00000    857.750000         2017-04-01 00:00:00   
50%      10000.500000     44.00000   1736.000000         2017-07-03 00:00:00   
75%      15000.250000     72.00000   2613.000000         2017-10-02 00:00:00   
max      20000.000000    100.00000   5034.000000         2017-12-30 00:00:00   
std       5773.647028     30.75359   1011.951046                         NaN   

       online_order    list_price  standard_cost  product_first_sold_date  
count  19640.000000  20000.000000   19803.000000             19803.000000  
mean       0.500458   1107.829449     556.046951             38199.776549  
min        0.000000     12.010000       7.210000   

Data Quality Report: Transactions

    1. Consistency
    2. Accuracy
    3. Completeness
    4. Relevancy

    1.1) Consistency: among numerical columns
        - data is consistent among numerical cols
            - no negative values in list_price col

    1.2) Consistency: among object columns
        - data is consistent among object cols
            - uniform strings in all of these columns

    2) Accuracy:
        - data types are accurate in all columns except for product_first_sold-date example) 42105

    3) Completeness:
        - online_order has 360 null values
        - brand has 197 null values
        - product_line 197 null values
        - product_class has 197 null values
        - product_size 197 null values
        - standard_cost 197 null values
        - product_first_sold_date 197 null values

            - not a coincidence that there are 197 null values for 6 columns
                - maybe 1 or multiple brands missing product data

    4) Relevancy:
        - observed that data in all cols are relevant to column name and sheet title




__________ new_customer_list Dataset __________

* use old customer list 'CustomerDemographic' to perform EDA
*

In [7]:
# Preliminary Data Validation
print(new_customer_list.head(), '\n\n')


  first_name  last_name  gender  past_3_years_bike_related_purchases  \
0    Chickie    Brister    Male                                   86   
1      Morly     Genery    Male                                   69   
2    Ardelis  Forrester  Female                                   10   
3     Lucine      Stutt  Female                                   64   
4    Melinda     Hadlee  Female                                   34   

                   DOB                   job_title job_industry_category  \
0           1957-07-12             General Manager         Manufacturing   
1           1970-03-22         Structural Engineer              Property   
2  1974-08-28 00:00:00      Senior Cost Accountant    Financial Services   
3           1979-01-28  Account Representative III         Manufacturing   
4           1965-09-21           Financial Analyst    Financial Services   

      wealth_segment deceased_indicator owns_car  ...  state    country  \
0      Mass Customer               

In [8]:
# Completeness

print(new_customer_list.info(), '\n\n')

print(new_customer_list.isnull().sum())

    # last_name: 29 nulls
    # DOB: 17 nulls
    # job_title: 106 nulls
    # job_industry_category: 165 nulls

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 23 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   first_name                           1000 non-null   object 
 1   last_name                            971 non-null    object 
 2   gender                               1000 non-null   object 
 3   past_3_years_bike_related_purchases  1000 non-null   int64  
 4   DOB                                  983 non-null    object 
 5   job_title                            894 non-null    object 
 6   job_industry_category                835 non-null    object 
 7   wealth_segment                       1000 non-null   object 
 8   deceased_indicator                   1000 non-null   object 
 9   owns_car                             1000 non-null   object 
 10  tenure                               1000 non-null   int64  
 11  address                        

In [19]:
# Consistency: among object cols

objects1 = ['first_name', 'last_name', 'gender', 'DOB', 'job_title', 'job_industry_category', 'wealth_segment', 'deceased_indicator', 'owns_car', 'address', 'state', 'country']

for column in objects1:
    print(new_customer_list[column].value_counts(), '\n\n')

    # at a glance object col values look clean



first_name
Rozamond     3
Dorian       3
Mandie       3
Inglebert    2
Ricki        2
            ..
Diego        1
Lucilia      1
Eddy         1
Caron        1
Sylas        1
Name: count, Length: 940, dtype: int64 


last_name
Sissel       2
Minshall     2
Borsi        2
Shoesmith    2
Sturch       2
            ..
O'Moylane    1
Axtens       1
Moxted       1
Conrad       1
Duffill      1
Name: count, Length: 961, dtype: int64 


gender
Female    513
Male      470
U          17
Name: count, dtype: int64 


DOB
1965-07-03    2
1978-01-15    2
1979-07-28    2
1995-08-13    2
1941-07-21    2
             ..
1978-05-27    1
1945-08-08    1
1943-08-27    1
1999-10-24    1
1955-10-02    1
Name: count, Length: 961, dtype: int64 


job_title
Associate Professor         15
Environmental Tech          14
Software Consultant         14
Chief Design Engineer       13
Assistant Manager           12
                            ..
Accountant II                1
Programmer IV                1
Adminis

In [9]:
# Consistency: among numerical cols

numeric_cols = ['past_3_years_bike_related_purchases', 'tenure', 'postcode', 'property_valuation', 'Rank', 'Value']

for column in numeric_cols:
    print(new_customer_list[column].value_counts(), '\n\n')

    # at a glance numerical cols look clean

past_3_years_bike_related_purchases
60    20
59    18
42    17
70    17
11    16
      ..
19     5
9      5
92     5
85     4
20     3
Name: count, Length: 100, dtype: int64 


tenure
9     79
13    74
11    68
10    63
12    61
5     60
7     60
17    59
15    58
8     55
14    54
16    49
6     45
4     36
18    36
19    34
3     26
21    24
20    22
2     15
22    12
1      8
0      2
Name: count, dtype: int64 


postcode
2145    9
2232    9
2750    7
3977    7
2148    7
       ..
2576    1
2479    1
3065    1
2762    1
4215    1
Name: count, Length: 522, dtype: int64 


property_valuation
9     173
8     162
7     138
10    116
6      70
11     62
5      57
4      53
3      51
12     46
2      42
1      30
Name: count, dtype: int64 


Rank
760     13
259     12
455      9
133      9
386      9
        ..
240      1
248      1
657      1
249      1
1000     1
Name: count, Length: 324, dtype: int64 


Value
0.637500    13
1.062500    12
0.892500     9
1.237500     9
0.945625     9
  

In [31]:
new_customer_list['DOB'] = pd.to_datetime(new_customer_list['DOB'], errors='coerce')

In [32]:
invalid_dates = new_customer_list['DOB'][new_customer_list['DOB'].isna()]

In [34]:
print(type(new_customer_list['DOB'][0]))

    # converted values in DOB col to timestamp

<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [38]:
out_of_range_dates = new_customer_list[(new_customer_list['DOB'] < min_date) | (new_customer_list['DOB'] > max_date)]

    # grouping all out of range dates

In [42]:
# Print the results
print("Invalid Dates:")
print(len(invalid_dates))

    # 17 invalid dates but remember that there are 17 nulls in this col

Invalid Dates:
17

Dates Out of Expected Range:
Empty DataFrame
Columns: [first_name, last_name, gender, past_3_years_bike_related_purchases, DOB, job_title, job_industry_category, wealth_segment, deceased_indicator, owns_car, tenure, address, postcode, state, country, property_valuation, Unnamed: 16, Unnamed: 17, Unnamed: 18, Unnamed: 19, Unnamed: 20, Rank, Value]
Index: []

[0 rows x 23 columns]


Data Quality Report: New_Customer_List

    1. Consistency
    2. Accuracy
    3. Completeness
    4. Relevancy

    1.1) Consistency: among numerical columns
        - data is consistent among numerical cols

    1.2) Consistency: among object columns
        - data is consistent among object cols
            - relatively uniform strings in all of these columns

    2) Accuracy:
        - DOB col is in object type when it should be datetime

    3) Completeness:
        - last_name: 29 nulls
        - DOB: 17 nulls
        - job_title: 106 nulls
        - job_industry_category: 165 nulls

    4) Relevancy:
        - column index 16 - 20 are titled 'Unnamed'
        - meaning of Rank and Value columns are unclear; no metadata provided


In [ ]:
_______ customer_demographic dataset ________

In [10]:
# preliminary data validation
print(customer_demographic.head())

   customer_id      first_name  last_name  gender  \
0            1         Laraine  Medendorp       F   
1            2             Eli    Bockman    Male   
2            3           Arlin     Dearle    Male   
3            4          Talbot        NaN    Male   
4            5  Sheila-kathryn     Calton  Female   

   past_3_years_bike_related_purchases                  DOB  \
0                                   93  1953-10-12 00:00:00   
1                                   81  1980-12-16 00:00:00   
2                                   61  1954-01-20 00:00:00   
3                                   33  1961-10-03 00:00:00   
4                                   56  1977-05-13 00:00:00   

                job_title job_industry_category     wealth_segment  \
0     Executive Secretary                Health      Mass Customer   
1  Administrative Officer    Financial Services      Mass Customer   
2      Recruiting Manager              Property      Mass Customer   
3                     

In [12]:
# Completeness
print(customer_demographic.isnull().sum())

customer_id                              0
first_name                               0
last_name                              125
gender                                   0
past_3_years_bike_related_purchases      0
DOB                                     87
job_title                              506
job_industry_category                  656
wealth_segment                           0
deceased_indicator                       0
default                                302
owns_car                                 0
tenure                                  87
dtype: int64


In [13]:
# Accuracy
print(customer_demographic.info(), '\n\n')

    # DOB is an object and should be datetime

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 13 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   customer_id                          4000 non-null   int64  
 1   first_name                           4000 non-null   object 
 2   last_name                            3875 non-null   object 
 3   gender                               4000 non-null   object 
 4   past_3_years_bike_related_purchases  4000 non-null   int64  
 5   DOB                                  3913 non-null   object 
 6   job_title                            3494 non-null   object 
 7   job_industry_category                3344 non-null   object 
 8   wealth_segment                       4000 non-null   object 
 9   deceased_indicator                   4000 non-null   object 
 10  default                              3698 non-null   object 
 11  owns_car                      

In [ ]:
# Relevancy

    # default column has mixed datatypes and I cannot identify what the column means by 'default' and what their values mean as there is no consistency here



In [16]:
# Consistency: numerical cols

numeric_cols1 = ['customer_id','past_3_years_bike_related_purchases','tenure']

for column in numeric_cols1:
    print(customer_demographic[column].value_counts(), '\n')

    # consistency in numerical columns


customer_id
1       1
2672    1
2659    1
2660    1
2661    1
       ..
1339    1
1340    1
1341    1
1342    1
4000    1
Name: count, Length: 4000, dtype: int64 

past_3_years_bike_related_purchases
16    56
19    56
67    54
20    54
2     50
      ..
8     28
95    27
85    27
86    27
92    24
Name: count, Length: 100, dtype: int64 

tenure
7.0     235
5.0     228
11.0    221
10.0    218
16.0    215
8.0     211
18.0    208
12.0    202
9.0     200
14.0    200
6.0     192
13.0    191
4.0     191
17.0    182
15.0    179
1.0     166
3.0     160
19.0    159
2.0     150
20.0     96
22.0     55
21.0     54
Name: count, dtype: int64 



In [19]:
# Consistency: object cols

objects2 = ['first_name', 'last_name', 'gender', 'DOB', 'job_title', 'job_industry_category', 'wealth_segment', 'deceased_indicator', 'owns_car' ]

for column in objects2:
    print(customer_demographic[column].value_counts(), '\n')

    # gender: inconsistency (should just be 'Female' / 'Male' or 'F' or 'M')

first_name
Max         5
Tobe        5
Timmie      5
Kippy       4
Pail        4
           ..
Katy        1
Hakim       1
Fanchon     1
Jeanette    1
Sarene      1
Name: count, Length: 3139, dtype: int64 

last_name
Pristnor     3
Ramsdell     3
Eager        2
Zisneros     2
Dredge       2
            ..
Whittock     1
Carwithen    1
Blas         1
Dearlove     1
Oldland      1
Name: count, Length: 3725, dtype: int64 

gender
Female    2037
Male      1872
U           88
F            1
Femal        1
M            1
Name: count, dtype: int64 

DOB
1978-01-30    7
1964-07-08    4
1962-12-17    4
1978-08-19    4
1977-05-13    4
             ..
1989-06-16    1
1998-09-30    1
1985-03-11    1
1989-10-23    1
1991-11-05    1
Name: count, Length: 3448, dtype: int64 

job_title
Business Systems Development Analyst    45
Tax Accountant                          44
Social Worker                           44
Internal Auditor                        42
Recruiting Manager                      41
    

In [14]:
print(customer_demographic.columns)

Index(['customer_id', 'first_name', 'last_name', 'gender',
       'past_3_years_bike_related_purchases', 'DOB', 'job_title',
       'job_industry_category', 'wealth_segment', 'deceased_indicator',
       'default', 'owns_car', 'tenure'],
      dtype='object')


Data Quality Report: customer_demographic

    1. Consistency
    2. Accuracy
    3. Completeness
    4. Relevancy

    1.1) Consistency: among numerical columns
        - consistency in numerical columns

    1.2) Consistency among object cols
        - gender: inconsistency (should just be 'Female' / 'Male' or 'F' or 'M')

    2) Accuracy:
        - DOB is an object type and should be datetime

    3) Completeness:

        - last_name                              125 nulls
        - DOB                                     87 nulls
        - job_title                              506 nulls
        - job_industry_category                  656 nulls


    4) Relevancy:
        - default column has mixed datatypes and I cannot identify what the column means by 'default' and what their values mean as there is no consistency here



_____________ customer_address dataset ___________

In [25]:
# taking a peek into the dataset
print(customer_address.head(20))

    customer_id                  address  postcode            state  \
0             1       060 Morning Avenue      2016  New South Wales   
1             2      6 Meadow Vale Court      2153  New South Wales   
2             4       0 Holy Cross Court      4211              QLD   
3             5      17979 Del Mar Point      2448  New South Wales   
4             6         9 Oakridge Court      3216              VIC   
5             7         4 Delaware Trail      2210  New South Wales   
6             8      49 Londonderry Lane      2650  New South Wales   
7             9          97736 7th Trail      2023  New South Wales   
8            11     93405 Ludington Park      3044              VIC   
9            12  44339 Golden Leaf Alley      4557              QLD   
10           13      2 Sutherland Street      3799              VIC   
11           14          9 Mcbride Trail      2760  New South Wales   
12           15   9861 New Castle Avenue      2428  New South Wales   
13    

In [21]:
# completeness

print(customer_address.isnull().sum())

    # no nulls in this entire dataset

customer_id           0
address               0
postcode              0
state                 0
country               0
property_valuation    0
dtype: int64


In [22]:
# accuracy

print(customer_address.info())

    # all data types are appropriate for cols

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3999 entries, 0 to 3998
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   customer_id         3999 non-null   int64 
 1   address             3999 non-null   object
 2   postcode            3999 non-null   int64 
 3   state               3999 non-null   object
 4   country             3999 non-null   object
 5   property_valuation  3999 non-null   int64 
dtypes: int64(3), object(3)
memory usage: 187.6+ KB
None


In [27]:
# relevancy

print(customer_address['property_valuation'].head(10))

    # value meaning in property_valuation is unclear
        # shortened to 10, 9, 4 etc

0    10
1    10
2     9
3     4
4     9
5     9
6     4
7    12
8     8
9     4
Name: property_valuation, dtype: int64


In [24]:
# consistency

print(customer_address['state'].value_counts())
      # NSW value contradicts with New South Wales
      # VIC value contradicts with Victoria
        # need to find a way to combine these since they mean the same thing

state
NSW                2054
VIC                 939
QLD                 838
New South Wales      86
Victoria             82
Name: count, dtype: int64


Data Quality Report: customer_address

    1. Consistency
    2. Accuracy
    3. Completeness
    4. Relevancy

    1.1) Consistency:
        - NSW value contradicts with New South Wales
        - VIC value contradicts with Victoria
        - need to find a way to combine these since they mean the same thing

    2) Accuracy:
        - all data types are appropriate for cols

    3) Completeness:
        - no nulls in this entire dataset

    4) Relevancy:
        - value meaning in property_valuation is unclear
        - values shortened / abbreviated to 10, 9, 4 etc


In [ ]:
# regression and clustering models
    # focus the most on clustering since groups

In [5]:
print(customer_demographic.head())

   customer_id      first_name  last_name  gender  \
0            1         Laraine  Medendorp       F   
1            2             Eli    Bockman    Male   
2            3           Arlin     Dearle    Male   
3            4          Talbot        NaN    Male   
4            5  Sheila-kathryn     Calton  Female   

   past_3_years_bike_related_purchases                  DOB  \
0                                   93  1953-10-12 00:00:00   
1                                   81  1980-12-16 00:00:00   
2                                   61  1954-01-20 00:00:00   
3                                   33  1961-10-03 00:00:00   
4                                   56  1977-05-13 00:00:00   

                job_title job_industry_category     wealth_segment  \
0     Executive Secretary                Health      Mass Customer   
1  Administrative Officer    Financial Services      Mass Customer   
2      Recruiting Manager              Property      Mass Customer   
3                     